# 개요
- SQLite + Python으로 사내식당 데이터와 날씨 데이터를 분석에 알맞는 데이터로 변환해 DB에 저장했습니다.
- csv 형태 데이터를 불러와 Python, Pandas로 불러와 원하는 형태로 변환하고 그 데이터를 SQLite DB에 저장하는 프로세스입니다.
    + csv -> python -> [transformat] -> SQLite -> python -> [Machinelearning] -> result의 프로세스를 거칩니다.
- 매주 예상 기온, 풍속, 강수량, 습도 등과 메뉴, 신메뉴 여부 등을 입력하면 자동으로 식수인원을 예측할 수 있도록 데이터 프로세스를 만들어봤습니다.

# 1. Import Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import matplotlib.font_manager as fm
import matplotlib as mpl
mpl.rcParams["axes.unicode_minus"] = False

#SQLite 관련 library import
import sqlite3
from os import mkdir
import csv

# 2. SQLite로 DB, 테이블 생성 후 데이터 입력하기

In [48]:
# 데이터를 읽는 함수 실행하려면 실행파일 옆에 data 폴더와 안에 train, weather 파일이 있어야합니다.
# 받아야할 변수는 2개 지정해주세요
# 예) data, weather = data_read
# 예) data, weather에 데이터 들어가 있어요
def data_read():
    data = pd.read_csv("data/data.csv", encoding='utf-8')
    weather = pd.read_csv("data/weather.csv", encoding='euc-kr')
    return data, weather

In [49]:
# 함수 실행하면 team3.db 생성과 함께 hr, lunch, dinner, weather 테이블이 만들어집니다.
def make_database():
    try:
        conn = sqlite3.connect('data/team3.db')
    except sqlite3.OperationalError:
        mkdir('folder_A')
    finally:
        conn = sqlite3.connect('data/team3.db')
        cur = conn.cursor()

    hr = """create table if not exists hr(
        datetime  datetime,
        worker_number   int,
        real_number int,
        biztrip_number  int,
        overtime_number int,
        telecom_number  int    
    )
    """
    cur.execute(hr)
    conn.commit()


    lunch = """create table if not exists lunch(
        datetime  datetime,
        new_lunch   varchar(10),
        lunch_rice varchar(20),
        lunch_soup  varchar(20),
        lunch_main varchar(20),
        lunch_number  int    
    )
    """
    cur.execute(lunch)
    conn.commit()


    dinner = """create table if not exists dinner(
        datetime  datetime,
        new_dinner   varchar(10),
        dinner_rice varchar(20),
        dinner_soup  varchar(20),
        dinner_main varchar(20),
        dinner_number  int    
    )
    """
    cur.execute(dinner)
    conn.commit()


    weather = """create table if not exists weather(
        datetime  datetime,
        temperature   float(32),
        rain float(32),
        wind float(32),
        humidity  float(32),
        discomfort_index    float(32),
        perceived_temperature   float(32)
    )
    """
    cur.execute(weather)
    conn.commit()

    calendar = """create table if not exists calendar(
        datetime  datetime,
        month   int,
        date int,
        weekdays  varchar(16),
        season varchar(16),
        vacation  int    
        )
        """
    cur.execute(calendar)
    conn.commit()


    conn.close()

In [50]:
# 사내식당 데이터, 날짜 데이터를 합쳐서 DB에 적합한 프레임 형태로 바꿔주는 함수.
# 데이터가 업데이트될 때마다 형태를 매번 바꿔주는건 비효율적인 일이라 판단해 함수로 만들어 업데이트될 때마다 사용할 수 있게 만들었습니다.
# data, weather를 결합해 하나의 데이터로 만들어줍니다. 변수 하나만 지정해주세요
# 예) df = transform(data, weather)

def transform(data, weather):
    #### 1. 일자를 datetime 형태로 변경
    data['일자'] = pd.to_datetime(data['일자'])
    #### 2. 실질 사내 근무자 수
    data['실질정원수'] = data['본사정원수'] - (data['본사휴가자수'] + data['본사출장자수'] + data['현본사소속재택근무자수'])
    data = data.loc[:, ['일자', '요일', '본사정원수', '실질정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '중식메뉴', '석식메뉴', '중식계', '석식계']]
    #### 3. 월, 일 칼럼과 계절 칼럼, 연휴 칼럼 만들기
    data['Month'] = data['일자'].dt.strftime('%m')
    data['Date'] = data['일자'].dt.strftime('%d')
    season = []

    for index in range(len(data)):
        if data['Month'][index] == '03' or data['Month'][index] == '04' or data['Month'][index] == '05':
            season.append('봄')
        elif data['Month'][index] == '06' or data['Month'][index] == '07' or data['Month'][index] == '08':
            season.append('여름')
        elif data['Month'][index] == '09' or data['Month'][index] == '10' or data['Month'][index] == '11':
            season.append('가을')
        elif data['Month'][index] == '12' or data['Month'][index] == '01' or data['Month'][index] == '02':
            season.append('겨울')
    
    data['Season'] = season



    holiday_gap=[]

    for i in range(len(data)):
        if i == len(data) - 1:
            holiday_gap.append(0)
        elif int((pd.to_datetime(data['일자'][i+1])-pd.to_datetime(data['일자'][i])).days)==1:
            holiday_gap.append(0)
        elif int((pd.to_datetime(data['일자'][i+1])-pd.to_datetime(data['일자'][i])).days)==2:
            holiday_gap.append(1)
        elif int((pd.to_datetime(data['일자'][i+1])-pd.to_datetime(data['일자'][i])).days)==3:
            holiday_gap.append(0)
        else:
            holiday_gap.append(1)
            
    data['연휴'] = holiday_gap



    #### 4. 신메뉴 여부 칼럼 만들기 Y = 신메뉴 / N = 신메뉴 X
    New_lunch = []
    New_dinner = []

    for index in range(len(data)):
        if 'New' in data['중식메뉴'][index]:
            New_lunch.append('Y')
        else:
            New_lunch.append('N')

    for index in range(len(data)):
        if 'New' in data['석식메뉴'][index]:
            New_dinner.append('Y')
        else:
            New_dinner.append('N')
    
    data['New_lunch'] = New_lunch
    data['New_dinner'] = New_dinner

    #### 5. 점심, 저녁에서 밥, 국, 메인반찬 칼럼 만들기
    #점심
    lunch = []
    for index in range(len(data)):
        tmp = data.loc[index,'중식메뉴'].split(' ') # 스페이스로 구분
        tmp = ' '.join(tmp).split()    # 빈칸 제거

        # ()안에 있는 내용 제거
        for menu in tmp:
            if '(' in menu:
                tmp.remove(menu)
        lunch.append(tmp)

    for index in range(len(data)):
        if '쌀밥' in lunch[index][0]:
            lunch[index][0] = '밥'
    
    rice=[]
    soup=[]
    main=[]

    for i in range(len(data)):
        if lunch[i][0]:
            rice.append(lunch[i][0])
        if lunch[i][1]:
            soup.append(lunch[i][1])
        if lunch[i][2]:
            main.append(lunch[i][2])

    data['lunch_rice'] = rice
    data['lunch_soup'] = soup
    data['lunch_main'] = main

    #저녁
    dinner = []

    for index in range(len(data)):
        tmp = data.loc[index,'석식메뉴'].split(' ')
        tmp = ' '.join(tmp).split()

        for menu_d in tmp:
            if '(' in menu_d:
                tmp.remove(menu_d)
        dinner.append(tmp)

    dinner_rice=[]
    dinner_soup=[]
    dinner_main=[]


    for word in dinner:
        
        
        if len(word) == 0:
            dinner_rice.append('None')
            dinner_soup.append('None')
            dinner_main.append('None')
        elif '*' in word:
            dinner_rice.append('None')
            dinner_soup.append('None')
            dinner_main.append('None')
        elif '가정의날' in word:
            dinner_rice.append('None') 
            dinner_soup.append('None')
            dinner_main.append('None')
        elif '가정의달' in word:
            dinner_rice.append('None') 
            dinner_soup.append('None')
            dinner_main.append('None')
        elif '자기계발의날' in word:
            dinner_rice.append('None') 
            dinner_soup.append('None')
            dinner_main.append('None')
        elif '*자기계발의날*' in word:
            dinner_rice.append('None') 
            dinner_soup.append('None')
            dinner_main.append('None')
        elif '자기개발의날' in word:
            dinner_rice.append('None') 
            dinner_soup.append('None')
            dinner_main.append('None')

        else:
            dinner_rice.append(word[0])
            dinner_soup.append(word[1])
            dinner_main.append(word[2])
    
    data['dinner_rice'] = dinner_rice
    data['dinner_soup'] = dinner_soup
    data['dinner_main'] = dinner_main

    for index in range(len(data)):
        if '쌀밥' in data['dinner_rice'][index]:
            data['dinner_rice'][index] = '밥'
        elif '흑미밥' in data['dinner_rice'][index]:
            data['dinner_rice'][index] = '밥'
        
    del data['중식메뉴']
    del data['석식메뉴']



    #### 6. 날짜 데이터 merge
    # merge 전 날짜 데이터 전처리
    weather['일자'] = pd.to_datetime(weather['일시'])
    del weather['일시']
    del weather['지점']
    del weather['지점명']

    #결측치 제거
    weather['일강수량(mm)']=weather['일강수량(mm)'].replace({np.NaN:0})
    weather['평균 상대습도(%)'] = weather['평균 상대습도(%)'].replace({np.NaN:0})

    #칼럼명 변경
    weather.columns = ['기온', '강수량', '풍속', '습도', '일자']

    # 불쾌지수, 체감온도 칼럼 생성
    weather['불쾌지수'] = 9/5 * weather['기온'] - 0.55 * (1-weather['습도']/100) * (9/5 * weather['기온'] - 26) + 32
    weather['체감온도'] = 13.12 + 0.6215 * weather['기온'] - 11.37 * (weather['풍속'] ** 0.16) + 0.3965 * (weather['풍속'] ** 0.16) * weather['기온']    #### 7. 칼럼명 영어로 바꾸기
    
    
    #### 7. 칼럼명 영어로 바꾸기
    df = pd.merge(data, weather, how='inner', on='일자')

    col_eng = ['datetime', 'weekdays', 'worker_number', 'real_number', 'vacation_number', 'biztrip_number', 'overtime_number', 'telecom_number', 'lunch_number',
    'dinner_number', 'month', 'date', 'season', 'vacation', 'new_lunch', 'new_dinner', 'lunch_rice', 'lunch_soup', 'lunch_main', 'dinner_rice', 'dinner_soup', 'dinner_main',
    'temperature', 'rain', 'wind', 'humidity', 'discomfort_index', 'perceived_temperature']

    df.columns=col_eng

    return df

In [51]:
# SQL 데이터 구조는 중복 X. 데이터 목적, 저장소에 맞춰 데이터를 나눠줬습니다.
# 사내 인사 정보, 날씨 정보, 사내식당 점심 정보, 사내식당 저녁 정보 넷으로 나눴습니다.

def split(data):
    hr_data = data.loc[:, ["datetime", "worker_number", "real_number", "biztrip_number", "overtime_number", "telecom_number"]]
    lunch_data = data.loc[:, ["datetime", "new_lunch", "lunch_rice", "lunch_soup", "lunch_main", "lunch_number"]]
    dinner_data = data.loc[:, ["datetime", "new_dinner", "dinner_rice", "dinner_soup", "dinner_main", "dinner_number"]]
    weather_data = data.loc[:, ["datetime", "temperature", 'rain', 'wind', 'humidity', 'discomfort_index', 'perceived_temperature']]
    calendar_data = data.loc[:, ['datetime', 'month', 'date', 'weekdays', 'season', 'vacation']]

    return hr_data, lunch_data, dinner_data, weather_data, calendar_data


In [52]:
# 데이터 입력하면 한번에 다 저장되도록 함수로 만들었습니다.
# 데이터를 저장하는 이유는 파이썬에서 db에 데이터를 입력하기 위해 다시 csv 라이브러리로 open 시켜 한줄씩 불러와 반복문으로 입력해야하기 때문입니다. 

def save(hr_data, lunch_data, dinner_data, weather_data, calendar_data):
    hr_data.to_csv("data/hr_data.csv", encoding='utf-8', index=False)
    lunch_data.to_csv("data/lunch_data.csv", encoding='utf-8', index=False)
    dinner_data.to_csv("data/dinner_data.csv", encoding='utf-8', index=False)
    weather_data.to_csv("data/weather_data.csv", encoding='utf-8', index=False)
    calendar_data.to_csv("data/calendar_data.csv", encoding='utf-8', index=False)

In [53]:
# 함수 호출로 csv 데이터를 db에 저장할 수 있게 만들었습니다.
def hr_db():
    conn = sqlite3.connect("data/team3.db")
    cur = conn.cursor()

    file = csv.reader(open("data/hr_data.csv", "r"), delimiter=",")
    next(file)

    for row in file:
        cur.execute("insert into hr values(?, ?, ?, ?, ?, ?)", row)

    conn.commit()
    conn.close()

In [54]:
def lunch_db():
    conn = sqlite3.connect("data/team3.db")
    cur = conn.cursor()

    file = csv.reader(open("data/lunch_data.csv", "r"), delimiter=",")
    next(file)

    for row in file:
        cur.execute("insert into lunch values(?, ?, ?, ?, ?, ?)", row)

    conn.commit()
    conn.close()


In [55]:
def dinner_db():
    conn = sqlite3.connect("data/team3.db")
    cur = conn.cursor()

    file = csv.reader(open("data/dinner_data.csv", "r"), delimiter=",")
    next(file)

    for row in file:
        cur.execute("insert into dinner values(?, ?, ?, ?, ?, ?)", row)

    conn.commit()
    conn.close()


In [56]:
def weather_db():
    conn = sqlite3.connect("data/team3.db")
    cur = conn.cursor()

    file = csv.reader(open("data/weather_data.csv", "r"), delimiter=",")
    next(file)

    for row in file:
        cur.execute("insert into weather values(?, ?, ?, ?, ?, ?, ?)", row)

    conn.commit()
    conn.close()


In [57]:
def calendar_db():
    conn = sqlite3.connect("data/team3.db")
    cur = conn.cursor()

    file = csv.reader(open("data/calendar_data.csv", "r"), delimiter=",")
    next(file)

    for row in file:
        cur.execute("insert into calendar values(?, ?, ?, ?, ?, ?)", row)

    conn.commit()
    conn.close()

In [58]:
# 반복 실행하지 않도록 주의할 것
# 반복 진행하면 데이터가 중복됩니다.

# 데이터베이스에 데이터 처음 입력할 때


data, weather = data_read()
make_database()
df = transform(data, weather)
hr_data, lunch_data, dinner_data, weather_data, calendar_data = split(df)
save(hr_data, lunch_data, dinner_data, weather_data, calendar_data)
hr_db()
lunch_db()
dinner_db()
weather_db()
calendar_db()

/var/folders/dx/ny9cjsmd1bb3r42zbp694xkh0000gn/T/ipykernel_2602/1264440053.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'
/var/folders/dx/ny9cjsmd1bb3r42zbp694xkh0000gn/T/ipykernel_2602/1264440053.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'


## checkpoint : 2022. 01. 07

- 이를 막기 위해선 일자에 key값을 주는 방법이 있고
- 다른 하나의 방법은 update용 함수 꾸러미(class?)를 하나 더 만드는 것이라 생각합니다.
- 일자 key로 정하면 아래 코드 진행해도 업데이트는 안되고 막힐 것으로 추측됩니다.
- 따라서, 일자 key로 지정해 반복 입력되는 것 막아주고
- 업데이트 함수 꾸러미 만들어서 앞으로 추가 확보될 데이터 업데이트 자동화 시켜줄 수 있도록 해야할 듯 합니다.

# 3. DB에서 데이터 조회해 ERD까지
- 1. 데이터를 쪼개놨습니다. 그래서, Join으로 데이터를 결합했습니다.
- 2. 파이썬에선 DB 데이터를 읽을 때 튜플로 나옵니다. 따라서, 행을 하나씩 뽑을 때마다 각 데이터 값을 리스트에 따로 담고 판다스 이용해 데이터프레임으로 만들어줬습니다.
- 3. ERD 주제 정하고, 거기에 맞춰서 쿼리문 작성하는 방식으로 진행했습니다.

In [3]:
conn = sqlite3.connect("data/team3.db")
print(conn)
conn.close()

In [109]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("update calendar set weekdays='금' where datetime = '2018-06-01'")
cur.execute("select * from calendar where datetime between '2018-05-15' and '2018-06-15'")

for row in cur:
    print(row)

('2018-05-15', 5, 15, '화', '봄', 0)
('2018-05-16', 5, 16, '수', '봄', 0)
('2018-05-17', 5, 17, '목', '봄', 0)
('2018-05-18', 5, 18, '금', '봄', 0)
('2018-05-21', 5, 21, '월', '봄', 1)
('2018-05-23', 5, 23, '수', '봄', 0)
('2018-05-24', 5, 24, '목', '봄', 0)
('2018-05-25', 5, 25, '금', '봄', 0)
('2018-05-28', 5, 28, '월', '봄', 0)
('2018-05-29', 5, 29, '화', '봄', 0)
('2018-05-30', 5, 30, '수', '봄', 0)
('2018-05-31', 5, 31, '목', '봄', 0)
('2018-06-01', 6, 1, '금', '여름', 0)
('2018-06-04', 6, 4, '월', '여름', 0)
('2018-06-05', 6, 5, '화', '여름', 1)
('2018-06-07', 6, 7, '목', '여름', 0)
('2018-06-08', 6, 8, '금', '여름', 0)
('2018-06-11', 6, 11, '월', '여름', 0)
('2018-06-12', 6, 12, '화', '여름', 1)
('2018-06-14', 6, 14, '목', '여름', 0)
('2018-06-15', 6, 15, '금', '여름', 0)


In [110]:
cur.execute("select * from calendar where datetime between '2018-05-15' and '2018-06-15'")

for row in cur:
    print(row)

conn.close()

('2018-05-15', 5, 15, '화', '봄', 0)
('2018-05-16', 5, 16, '수', '봄', 0)
('2018-05-17', 5, 17, '목', '봄', 0)
('2018-05-18', 5, 18, '금', '봄', 0)
('2018-05-21', 5, 21, '월', '봄', 1)
('2018-05-23', 5, 23, '수', '봄', 0)
('2018-05-24', 5, 24, '목', '봄', 0)
('2018-05-25', 5, 25, '금', '봄', 0)
('2018-05-28', 5, 28, '월', '봄', 0)
('2018-05-29', 5, 29, '화', '봄', 0)
('2018-05-30', 5, 30, '수', '봄', 0)
('2018-05-31', 5, 31, '목', '봄', 0)
('2018-06-01', 6, 1, '금', '여름', 0)
('2018-06-04', 6, 4, '월', '여름', 0)
('2018-06-05', 6, 5, '화', '여름', 1)
('2018-06-07', 6, 7, '목', '여름', 0)
('2018-06-08', 6, 8, '금', '여름', 0)
('2018-06-11', 6, 11, '월', '여름', 0)
('2018-06-12', 6, 12, '화', '여름', 1)
('2018-06-14', 6, 14, '목', '여름', 0)
('2018-06-15', 6, 15, '금', '여름', 0)


## (1) 코로나 이전 이후, 재택근무자 현황
+ 재택근무자가 50명 초과가 된 시점을 기준
+ 그 날짜를 기준으로 하여 재택근무자 수의 최대값, 최소값, 평균을 구하자

In [28]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("select datetime, telecom_number from hr where telecom_number > 50") 
#2020.3.20부터 본격적으로 코로나 영향으로 대규모 재택근무 시작

for row in cur:
    print(row)

conn.close()

('2020-03-20', 135)
('2020-03-23', 285)
('2020-03-24', 303)
('2020-03-25', 303)
('2020-03-26', 288)
('2020-03-27', 260)
('2020-03-30', 488)
('2020-03-31', 533)
('2020-04-01', 420)
('2020-04-02', 395)
('2020-04-03', 340)
('2020-04-06', 477)
('2020-04-07', 490)
('2020-04-08', 479)
('2020-04-09', 488)
('2020-04-10', 417)
('2020-04-13', 447)
('2020-04-14', 485)
('2020-04-16', 447)
('2020-04-17', 389)
('2020-04-20', 471)
('2020-04-21', 450)
('2020-04-22', 443)
('2020-04-23', 437)
('2020-04-24', 387)
('2020-04-27', 418)
('2020-04-28', 410)
('2020-04-29', 396)
('2020-05-04', 189)
('2020-05-06', 291)
('2020-05-07', 285)
('2020-05-08', 246)
('2020-05-11', 245)
('2020-05-12', 250)
('2020-05-13', 236)
('2020-05-14', 212)
('2020-05-15', 201)
('2020-05-18', 246)
('2020-05-19', 237)
('2020-05-20', 258)
('2020-05-21', 218)
('2020-05-22', 206)
('2020-05-25', 229)
('2020-05-26', 200)
('2020-05-27', 235)
('2020-05-28', 212)
('2020-05-29', 202)
('2020-06-01', 215)
('2020-06-02', 213)
('2020-06-03', 191)


In [24]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("select max(telecom_number), min(telecom_number), avg(telecom_number) from hr where datetime < '2020-03-20'")

for row in cur:
    print(row)

conn.close()

#코로나 이전에는 많아야 25명 정도 재택근무

(25, 0, 0.2900990099009901)


In [29]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("select max(telecom_number), min(telecom_number), avg(telecom_number) from hr where datetime >= '2020-03-20'")

for row in cur:
    print(row)

conn.close()

#코로나 이후에는 늘 평균 267명 정도의 사원이 재택근무

(533, 95, 267.34358974358975)


## (2) 코로나 이전 이후, 구내식당 이용인원(중식, 석식)
+ 2020년 3월 20일 이전과 이후로 나눠서 살펴보자
+ 이용인원의 최대값, 최소값, 평균을 구하자

In [33]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("select max(lunch_number), min(lunch_number),avg(lunch_number) from lunch where datetime < '2020-03-20'")

for row in cur:
    print(row)

conn.close()

(1459, 296, 891.2029702970297)


In [35]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("select max(dinner_number), min(dinner_number),avg(dinner_number) from dinner where datetime < '2020-03-20' and dinner_number != 0")

for row in cur:
    print(row)

conn.close()

# 중식은 늘 운영하지만, 석식은 운영하지 않는 날이 더러 있으므로, 0인 데이터를 제외해보자

(905, 104, 490.26756198347107)


In [36]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("select max(lunch_number), min(lunch_number),avg(lunch_number) from lunch where datetime >= '2020-03-20'")

for row in cur:
    print(row)

conn.close()

# 평균이 코로나 이전 대비 6명 정도 줄었는데 유의미한 변화인가?

(1405, 358, 885.8358974358974)


In [37]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("select max(dinner_number), min(dinner_number),avg(dinner_number) from dinner where datetime >= '2020-03-20' and dinner_number != 0")

for row in cur:
    print(row)

conn.close()

# 중식은 늘 운영하지만, 석식은 운영하지 않는 날이 더러 있으므로, 0인 데이터를 제외해보자
# 코로나 이전 대비 평균이 69명정도 줄었는데 유의미한 변화인가?

(754, 123, 421.94329896907215)


## (3) 메뉴에 따른 이용인원

+ 일반 밥이냐 그 외 밥(볶음밥, 비빔밥, 특수메뉴)이냐의 차이
    * 일반 밥은 DB에 '밥'이라고 기록, 나머지는 그 메뉴의 이름이 써있음
    
+ 국/찌개/탕의 차이
    * 뒤 글자와 like연산자 사용해서 국이냐 찌개냐 탕이냐 구분

In [42]:
# 일반 밥 vs 특수 밥(혹은 면)
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("SELECT count(lunch_number), avg(lunch_number), max(lunch_number), min(lunch_number) FROM lunch WHERE lunch_rice = '밥'")

for row in cur:
    print(row)

conn.close()

(1003, 898.2542372881356, 1459, 296)


In [43]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("SELECT count(lunch_number), avg(lunch_number), max(lunch_number), min(lunch_number) FROM lunch WHERE lunch_rice != '밥'")

for row in cur:
    print(row)

conn.close()

# 일반 밥이 나오는 날이 더 자주 있기도 하지만 이용인원에 대한 평균값과 최대값이 크게 나타난다.(표준편차를 구할 필요도 있음)

(202, 851.009900990099, 1319, 387)


In [44]:
# 국 vs 찌개 vs 탕
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("SELECT count(lunch_number), avg(lunch_number), max(lunch_number), min(lunch_number) FROM lunch WHERE lunch_soup LIKE '%국'")

for row in cur:
    print(row)

conn.close()


(806, 899.4937965260546, 1459, 296)


In [45]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("SELECT count(lunch_number), avg(lunch_number), max(lunch_number), min(lunch_number) FROM lunch WHERE lunch_soup LIKE '%찌개'")

for row in cur:
    print(row)

conn.close()

(222, 888.2342342342342, 1372, 389)


In [46]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("SELECT count(lunch_number), avg(lunch_number), max(lunch_number), min(lunch_number) FROM lunch WHERE lunch_soup LIKE '%탕'")

for row in cur:
    print(row)

conn.close()

# 데이터가 많은 순(국, 찌개, 탕)으로 평균값도 크다, 그렇지만 편차도 큰 것 같다.

(115, 864.0434782608696, 1292, 408)


## (4) 요일에 따른 구내 식당 이용 인원
+ group by weekdays를 사용

In [111]:
# 요일별 중식 인원(평균, 최대, 최소)

conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT weekdays, round(avg(lunch_number)), max(lunch_number), min(lunch_number) 
FROM calendar LEFT OUTER JOIN lunch ON calendar.datetime = lunch.datetime group by weekdays""")

for row in cur:
    print(row)

conn.close()

('금', 654.0, 1215, 387)
('목', 824.0, 1214, 311)
('수', 905.0, 1336, 571)
('월', 1144.0, 1459, 296)
('화', 926.0, 1324, 349)


In [112]:
# 요일별 석식 인원(평균, 최대, 최소)-단, 석식을 운영하지 않은 날은 고려되지 않도록

conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT weekdays, round(avg(dinner_number)), max(dinner_number), min(dinner_number) 
FROM calendar LEFT OUTER JOIN dinner ON calendar.datetime = dinner.datetime 
where dinner_number != 0 group by weekdays""")

for row in cur:
    print(row)

conn.close()

('금', 408.0, 755, 171)
('목', 480.0, 743, 117)
('수', 437.0, 905, 123)
('월', 539.0, 772, 104)
('화', 522.0, 902, 176)


## (5) 계절별 구내식당 이용 평균인원

In [54]:
# 계절별 중식 인원
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT season, round(avg(lunch_number)), max(lunch_number), min(lunch_number)
FROM calendar LEFT OUTER JOIN lunch ON calendar.datetime = lunch.datetime
group by season""")

for row in cur:
    print(row)

conn.close()

# 겨울과 봄에 이용인원이 많다, 밖에 나가기 번거롭기 때문?

('가을', 889.0, 1363, 358)
('겨울', 915.0, 1459, 296)
('봄', 903.0, 1362, 383)
('여름', 855.0, 1292, 389)


In [56]:
# 계절별 석식 인원(역시, 운영하지 않는 날의 데이터를 제외한다)
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT season, round(avg(dinner_number)), max(dinner_number), min(dinner_number)
FROM calendar LEFT OUTER JOIN dinner ON calendar.datetime = dinner.datetime
WHERE dinner_number != 0 group by season""")

for row in cur:
    print(row)

conn.close()

# 석식은 조금 다르다, 봄에는 많지만 겨울에는 그다지 많지 않다

('가을', 480.0, 905, 123)
('겨울', 466.0, 776, 104)
('봄', 506.0, 771, 123)
('여름', 464.0, 729, 212)


## (6) 휴일/연휴 전에 구내식당을 많이 이용하지 않는가?
+ 팀 내에서 추측하기를 "연휴 전에는 직원들이 회사 밖에서 식사를 하지 않을까?" 하여 해당 변수가 유효한지 보자
+ vacation 변수는 0이면 연휴 전이 아니란 뜻, 1이면 다음날부터 휴일이란 뜻

In [57]:
# 중식
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT vacation, round(avg(lunch_number)), max(lunch_number), min(lunch_number)
FROM calendar LEFT OUTER JOIN lunch ON calendar.datetime = lunch.datetime
group by vacation""")

for row in cur:
    print(row)

conn.close()

# 휴가가 다가오면 수치적으로 적어진 것은 사실이지만, 유의미한 변화인가?

(0, 898.0, 1459, 311)
(1, 711.0, 1241, 296)


In [58]:
# 석식
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT vacation, round(avg(dinner_number)), max(dinner_number), min(dinner_number)
FROM calendar LEFT OUTER JOIN dinner ON calendar.datetime = dinner.datetime
WHERE dinner_number != 0 group by vacation""")

for row in cur:
    print(row)

conn.close()

# 휴가가 다가오면 석식 이용인원에도 영향을 주는 것 같다, 차이가 크다.

(0, 485.0, 905, 124)
(1, 345.0, 743, 104)


## (7) 기온에 따른 구내식당 이용
+ 너무 덥거나, 너무 추우면 안에 있지 않을까?
+ 영상 4도 미만, 영상 25도 초과, 나머지의 경우를 나눠서 보자

In [71]:
# 평균 기온이 추운날
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT count(weather.datetime), avg(lunch_number), avg(temperature)
FROM weather LEFT OUTER JOIN lunch
ON weather.datetime = lunch.datetime
WHERE temperature < 4""")

for row in cur:
    print(row)

conn.close()

(230, 909.1304347826087, 0.4291304347826088)


In [74]:
# 평균 기온이 더운날
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT count(weather.datetime), avg(lunch_number), avg(temperature)
FROM weather LEFT OUTER JOIN lunch
ON weather.datetime = lunch.datetime
WHERE temperature > 25""")

for row in cur:
    print(row)

conn.close()

(156, 842.2371794871794, 27.23012820512821)


In [76]:
# 나머지
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT count(weather.datetime), avg(lunch_number), avg(temperature)
FROM weather LEFT OUTER JOIN lunch
ON weather.datetime = lunch.datetime
WHERE temperature between 4 and 25""")

for row in cur:
    print(row)

conn.close()

(819, 894.2173382173382, 14.804517704517716)


In [81]:
# 추운날
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT count(weather.datetime), avg(dinner_number), avg(temperature)
FROM weather LEFT OUTER JOIN dinner
ON weather.datetime = dinner.datetime
WHERE temperature < 4 and dinner_number != 0""")

for row in cur:
    print(row)

conn.close()

(223, 461.5112107623318, 0.4457399103139014)


In [82]:
# 더운날
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT count(weather.datetime), avg(dinner_number), avg(temperature)
FROM weather LEFT OUTER JOIN dinner
ON weather.datetime = dinner.datetime
WHERE temperature > 25 and dinner_number != 0""")

for row in cur:
    print(row)

conn.close()

(152, 463.29605263157896, 27.218421052631577)


In [83]:
# 나머지
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT count(weather.datetime), avg(dinner_number), avg(temperature)
FROM weather LEFT OUTER JOIN dinner
ON weather.datetime = dinner.datetime
WHERE temperature between 4 and 25 and dinner_number != 0""")

for row in cur:
    print(row)

conn.close()

(787, 486.78271918678524, 14.85298602287168)


## (8) 비가 오면 구내식당 이용인원이 많아질까?
+ 비가 오면 밖으로 나가기가 번거롭기 때문에 구내식당 이용이 많아질 수 있다고 추측가능

In [85]:
# 중식 (비가 온 날 vs 비가 안 온 날)
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT count(weather.datetime), avg(lunch_number)
FROM weather LEFT OUTER JOIN lunch
ON weather.datetime = lunch.datetime
WHERE rain > 0""")

for row in cur:
    print(row)

conn.close()

(309, 924.0291262135922)


In [87]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT count(weather.datetime), avg(lunch_number)
FROM weather LEFT OUTER JOIN lunch
ON weather.datetime = lunch.datetime
WHERE rain = 0""")

for row in cur:
    print(row)

conn.close()

# 수치적으로는 비가 온날 이용인원이 많지만 유의미한지는 알아봐야 한다

(896, 878.7142857142857)


In [88]:
# 석식 (비가 온 날 vs 비가 안 온 날)
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT count(weather.datetime), avg(dinner_number)
FROM weather LEFT OUTER JOIN dinner
ON weather.datetime = dinner.datetime
WHERE rain > 0""")

for row in cur:
    print(row)

conn.close()

(309, 463.1618122977346)


In [89]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""SELECT count(weather.datetime), avg(dinner_number)
FROM weather LEFT OUTER JOIN dinner
ON weather.datetime = dinner.datetime
WHERE rain = 0""")

for row in cur:
    print(row)

conn.close()

# 큰 변화는 없는 것 같다

(896, 461.2935267857143)


## (9) 신메뉴가 있는 날에 이용인원이 많아질까?

+ 가끔 식단에 New라는 표시로 신메뉴가 있음을 알린다
+ 그래서 New 칼럼을 넣어서 구분하였다
+ 그러나 데이터가 많이 없는 것도 있고, 신메뉴가 있다고 해서 평균 이용인원이 늘었다고 말할 수도 없다

In [98]:
# 중식
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""select count(*), avg(lunch_number), max(lunch_number) from lunch where new_lunch = 'Y'""")

for row in cur:
    print(row)

conn.close()

(26, 941.9230769230769, 1329)


In [99]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""select count(*), avg(lunch_number), max(lunch_number) from lunch where new_lunch = 'N'""")

for row in cur:
    print(row)

conn.close()

(1179, 889.1967769296014, 1459)


In [96]:
# 석식
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""select count(*), avg(dinner_number), max(dinner_number) from dinner where new_dinner = 'Y' and dinner_number != 0""")

for row in cur:
    print(row)

conn.close()

(21, 459.4761904761905, 712)


In [97]:
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""select count(*), avg(dinner_number), max(dinner_number) from dinner where new_dinner = 'N' and dinner_number != 0""")

for row in cur:
    print(row)

conn.close()

(1141, 479.2173531989483, 905)


# 데이터 합치기

In [2]:
cnt = 0
lndf = []
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""
SELECT * FROM lunch INNER JOIN hr USING (datetime)INNER JOIN weather USING (datetime)INNER JOIN calendar USING (datetime);""")

for row in cur:
    cnt += 1
    lndf.append(row)
    
conn.close()
print(cnt)
lndf = pd.DataFrame(lndf)
lndf

1205


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,2016-02-01,N,밥,오징어찌개,쇠불고기,1039,2601,2401,150,238,...,0.0,1.3,43.9,39.275534,0.641548,2,1,월,겨울,0
1,2016-02-02,N,밥,김치찌개,가자미튀김,867,2601,2378,173,319,...,0.0,0.8,47.5,36.562925,-0.160634,2,2,화,겨울,0
2,2016-02-03,N,카레덮밥,팽이장국,치킨핑거,1017,2601,2365,180,111,...,0.0,0.6,57.1,35.796707,0.964631,2,3,수,겨울,0
3,2016-02-04,N,밥,쇠고기무국,주꾸미볶음,978,2601,2277,220,355,...,0.0,0.7,53.4,38.396068,2.181494,2,4,목,겨울,0
4,2016-02-05,N,밥,떡국,돈육씨앗강정,925,2601,2142,181,34,...,0.0,1.0,44.4,41.575228,3.0734,2,5,금,겨울,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,N,밥,아욱국,수제함박스테이크,1093,2983,2319,198,4,...,0.0,0.5,64.0,36.57056,2.553014,1,20,수,겨울,0
1201,2021-01-21,N,밥,냉이된장국,동파육,832,2983,2309,231,462,...,1.7,0.6,77.8,43.391744,7.774119,1,21,목,겨울,0
1202,2021-01-22,N,전주비빔밥*약고추장,계란파국,요거닭,579,2983,2177,248,1,...,1.2,0.5,97.5,44.78425,9.778207,1,22,금,겨울,0
1203,2021-01-25,N,밥,전주식콩나물해장국,돈육간장불고기,1145,2983,2396,153,616,...,0.0,1.0,75.8,45.291246,8.1634,1,25,월,겨울,0


In [3]:
columns1 = ["datetime", "new_ln", "ln_rice", "ln_soup", "ln_main", "ln_eater",
           "workers", "real-in-worker", "biztrip", "overtime", "remote-work",
          "temperature", "rain", "wind", "humidity", "discomfort_index", "perceived-temperature", 
           "month", "date", "weekdays", "season", "near-holiday"]

lndf.columns = columns1
lndf

lndf.to_csv("../data/lunch_training.csv", encoding = "utf-8", index=False)

In [4]:
cnt = 0
dndf = []
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""
SELECT * FROM dinner INNER JOIN hr USING (datetime)INNER JOIN weather USING (datetime)INNER JOIN calendar USING (datetime)""")

for row in cur:
    cnt += 1
    dndf.append(row)
    
conn.close()
print(cnt)
dndf = pd.DataFrame(dndf)
dndf

1205


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,2016-02-01,N,밥,육개장,자반고등어구이,331,2601,2401,150,238,...,0.0,1.3,43.9,39.275534,0.641548,2,1,월,겨울,0
1,2016-02-02,N,콩나물밥*양념장,어묵국,유산슬,560,2601,2378,173,319,...,0.0,0.8,47.5,36.562925,-0.160634,2,2,화,겨울,0
2,2016-02-03,N,밥,청국장찌개,황태양념구이,573,2601,2365,180,111,...,0.0,0.6,57.1,35.796707,0.964631,2,3,수,겨울,0
3,2016-02-04,N,미니김밥*겨자장,우동,멕시칸샐러드,525,2601,2277,220,355,...,0.0,0.7,53.4,38.396068,2.181494,2,4,목,겨울,0
4,2016-02-05,N,밥,차돌박이찌개,닭갈비,330,2601,2142,181,34,...,0.0,1.0,44.4,41.575228,3.0734,2,5,금,겨울,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,N,김치볶음밥,미니쫄우동*맛살튀김,브로콜리깨소스무침,421,2983,2319,198,4,...,0.0,0.5,64.0,36.57056,2.553014,1,20,수,겨울,0
1201,2021-01-21,N,밥,쇠고기무국,삼치양념구이,353,2983,2309,231,462,...,1.7,0.6,77.8,43.391744,7.774119,1,21,목,겨울,0
1202,2021-01-22,N,밥,수제비국,수제맛쵸킹탕수육,217,2983,2177,248,1,...,1.2,0.5,97.5,44.78425,9.778207,1,22,금,겨울,0
1203,2021-01-25,N,밥,열무된장국,장어강정*데리야끼소스,502,2983,2396,153,616,...,0.0,1.0,75.8,45.291246,8.1634,1,25,월,겨울,0


In [5]:
columns2 = ["datetime", "new_dn", "dn_rice", "dn_soup", "dn_main", "dn_eater",
           "workers", "real-in-worker", "biztrip", "overtime", "remote-work",
          "temperature", "rain", "wind", "humidity", "discomfort_index", "perceived-temperature", 
           "month", "date", "weekdays", "season", "near-holiday"]

dndf.columns = columns2
dndf

dndf.to_csv("../data/dinner_training.csv", encoding = "utf-8", index=False)

In [6]:
cnt = 0
lndndf = []
conn = sqlite3.connect("data/team3.db")
cur = conn.cursor()

cur.execute("""
SELECT * FROM lunch INNER JOIN dinner USING (datetime)INNER JOIN hr USING (datetime)
INNER JOIN weather USING (datetime)INNER JOIN calendar USING (datetime)""")

for row in cur:
    cnt += 1
    lndndf.append(row)
    
conn.close()
print(cnt)
lndndf = pd.DataFrame(lndndf)
lndndf

1205


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,2016-02-01,N,밥,오징어찌개,쇠불고기,1039,N,밥,육개장,자반고등어구이,...,0.0,1.3,43.9,39.275534,0.641548,2,1,월,겨울,0
1,2016-02-02,N,밥,김치찌개,가자미튀김,867,N,콩나물밥*양념장,어묵국,유산슬,...,0.0,0.8,47.5,36.562925,-0.160634,2,2,화,겨울,0
2,2016-02-03,N,카레덮밥,팽이장국,치킨핑거,1017,N,밥,청국장찌개,황태양념구이,...,0.0,0.6,57.1,35.796707,0.964631,2,3,수,겨울,0
3,2016-02-04,N,밥,쇠고기무국,주꾸미볶음,978,N,미니김밥*겨자장,우동,멕시칸샐러드,...,0.0,0.7,53.4,38.396068,2.181494,2,4,목,겨울,0
4,2016-02-05,N,밥,떡국,돈육씨앗강정,925,N,밥,차돌박이찌개,닭갈비,...,0.0,1.0,44.4,41.575228,3.0734,2,5,금,겨울,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,N,밥,아욱국,수제함박스테이크,1093,N,김치볶음밥,미니쫄우동*맛살튀김,브로콜리깨소스무침,...,0.0,0.5,64.0,36.57056,2.553014,1,20,수,겨울,0
1201,2021-01-21,N,밥,냉이된장국,동파육,832,N,밥,쇠고기무국,삼치양념구이,...,1.7,0.6,77.8,43.391744,7.774119,1,21,목,겨울,0
1202,2021-01-22,N,전주비빔밥*약고추장,계란파국,요거닭,579,N,밥,수제비국,수제맛쵸킹탕수육,...,1.2,0.5,97.5,44.78425,9.778207,1,22,금,겨울,0
1203,2021-01-25,N,밥,전주식콩나물해장국,돈육간장불고기,1145,N,밥,열무된장국,장어강정*데리야끼소스,...,0.0,1.0,75.8,45.291246,8.1634,1,25,월,겨울,0


In [7]:
columns3 = ["datetime", "new_ln", "ln_rice", "ln_soup", "ln_main", "ln_eater", 
            "new_dn", "dn_rice", "dn_soup", "dn_main", "dn_eater",
           "workers", "real-in-worker", "biztrip", "overtime", "remote-work",
          "temperature", "rain", "wind", "humidity", "discomfort_index", "perceived-temperature", 
           "month", "date", "weekdays", "season", "near-holiday"]

lndndf.columns = columns3
lndndf

lndndf.to_csv("../data/lunch_dinner_training.csv", encoding = "utf-8", index=False)